# Image Generation

Before running this notebook, tou need to store your stability.ai API key in the environment variable `STABILITY_API_KEY`.

In [1]:
import os
import requests
from typing import Literal
from pathlib import Path

In [9]:
def getenv_or_except(key: str) -> str:
    val = os.getenv(key)
    if val is None:
        raise KeyError(f"'{key}' not found in OS environment variables.")
    return val

In [7]:
def send_generation_request(
    host,
    params,
):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {getenv_or_except('STABILITY_API_KEY')}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

In [8]:
prompt: str = "2x2 grid of photos of desertification in various parts of the world, photorealistic" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
aspect_ratio: Literal["21:9", "16:9", "3:2", "5:4", "1:1", "4:5", "2:3", "9:16", "9:21"] = "1:1" #@param ["21:9", "16:9", "3:2", "5:4", "1:1", "4:5", "2:3", "9:16", "9:21"]
seed = 0 #@param {type:"integer"}
output_format: Literal["webp", "jpeg", "png"] = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/generate/core"

params = {
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
    "aspect_ratio" : aspect_ratio,
    "seed" : seed,
    "output_format": output_format
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")

# Save and display result
generated = Path("gfx")/"images"/f"generated_{seed}.{output_format}"
with open(generated, "wb") as f:
    f.write(output_image)
print(f"Saved image {generated}")

KeyError: "'STABILITY_API_KEY' not found in OS environment variables."